In [3]:
import pandas as pd
import numpy as np

In [4]:
train = pd.read_csv('C:/Users/593939\Desktop/DSB 2019/DSB19-Team-Bubble-Bath/data/train.csv')
train_labels = pd.read_csv('C:/Users/593939\Desktop/DSB 2019/DSB19-Team-Bubble-Bath/data/train_labels.csv')
test = pd.read_csv('C:/Users/593939\Desktop/DSB 2019/DSB19-Team-Bubble-Bath/data/test.csv')
sample_submission = pd.read_csv('C:/Users/593939\Desktop/DSB 2019/DSB19-Team-Bubble-Bath/data/sample_submission.csv')

In [5]:
train.shape

(11341042, 11)

### Data Cleaning

Ya don't need 'em if they didn't take an Assessment!

In [6]:
assessed_ids = train[train['type'] == 'Assessment']['installation_id'].unique()
train = train[train['installation_id'].isin(assessed_ids)]
train.shape

(8294138, 11)

Ya don't need 'em if they ain't got no labels!

In [7]:
labeled_ids = train_labels['installation_id'].unique()
train = train[train['installation_id'].isin(labeled_ids)]
train.shape

(7734558, 11)

Turn `timestamp` into a datetime (for sorting purposes, just to be safe), and sort

In [8]:
train['timestamp'] = pd.to_datetime(train['timestamp'])
train.sort_values(['installation_id', 'timestamp'], inplace=True)

***

Notes: 
- not all assessments w/in a single `installation_id` have labels 
- in the test set, for each `installation_id` a random assessment is picked and you have to evaluate that one, so you can only use data prior to that assessment to make a prediction
- when training, make features out of all data prior to an assessment (if assessment has a label), then attach the label of that assessment to make `X` and `y`.
    - think: "what would I have guessed if the data was cut at this assessment"
- therefore, you can make multiple predictions per `installation_id`, one prediction per combination of `installation_id` and `game_session` (only if game_session has a label)

- every assessment begins with `event_code == 2000` and `type == Assessment`, so this is where we need to cut the data off

challenge: we need to evaluate the data for each `installation_id+game_session` pair for each cut
- this is going to take a long ass time
    - how can we do it quicker?

merge the `train` df w/ the `train_labels` df b/c we want to make sure we cut only where there is a label

In [9]:
train_raw = train.copy()  # just in case we need the original data

train = pd.merge(train, train_labels, on=['installation_id', 'game_session'], how='left')

filter the `train` data using a boolean mask, this is what we'll loop through (slowly)

In [10]:
train_cuts = train[(train['event_code'] == 2000) & 
                   (train['type'] == 'Assessment') & 
                   (train['accuracy_group'].notnull())]

how many loop iterations? count the combos of `installation_id` and `game_session`

In [11]:
combos = train_cuts['installation_id'] + train_cuts['game_session']
combos.nunique(), combos.count()  # make sure there are no duplicates

(17690, 17690)

### Event Codes

In [12]:
a = train[["installation_id", "event_code"]]

a1 = a.pivot_table(index='installation_id',
               columns='event_code',
               aggfunc=len,
               fill_value=0)

In [13]:
a1 = a1[[2010, 2020,2025, 2030, 2035,3010, 3020, 3021,3110, 3120, 3121,4020, 4025, 4030,4035, 4040, 4070,4080, 4090,4100,4110]]

In [14]:
a1["installation_id"] = a1.index

In [15]:
train = pd.merge(train, a1, on=['installation_id'], how='left')

### Time of Day

In [16]:
train["time_of_day"] = train["timestamp"].astype(str).str[11:13]
train["time_of_day"] = train["time_of_day"].astype(int)

In [17]:
train["segment_of_day"] = np.where(train["time_of_day"]<7, 1.524239, np.where(train["time_of_day"]<12, 1.746823, np.where(train["time_of_day"]<18, 1.556186, 1.502395)))


In [18]:
#train.groupby('segment_of_day')['accuracy_group'].mean()

In [19]:
# only about .04 correlation
# but if look at these per installation id there is somewhat of a trend

In [20]:
#train.groupby(['installation_id', "segment_of_day"])['accuracy_group'].mean().head(50)

### Time of day compared to usual

In [21]:
#Done in features function below

In [22]:
#a = train.groupby("installation_id").describe()
#avg_time = pd.DataFrame(data=a["time_of_day"]["mean"])
#avg_time.reset_index(level=0, inplace=True)

In [23]:

#train = pd.merge(train, avg_time, on=['installation_id'], how='left')

In [24]:
#train["time_diff"]=train["time_of_day"]-train["mean"]

In [25]:
def time_compared_to_normal(df):
    a = df.groupby("installation_id").describe()
    avg_time = pd.DataFrame(data=a["time_of_day"]["mean"])
    avg_time.reset_index(level=0, inplace=True)
    df = pd.merge(df, avg_time, on=['installation_id'], how='left')
    # calculate the "normal" time for this user to play
    return df["mean"] - df.iloc[-1,:]['time_of_day'] 

### Type of assessment

In [26]:
#make a column holding the values for the assessment as train.groupby('title_y')['accuracy_group'].mean()

In [27]:
train["assessment"] = np.where(train["title_y"]=="Bird Measurer (Assessment)", 1.14, np.where(train["title_y"]=="Cart Balancer (Assessment)", 1.86, np.where(train["title_y"]=="Cauldron Filler (Assessment)", 2.08, np.where(train["title_y"]=="Chest Sorter (Assessment)", 0.67, np.where(train["title_y"]=="Bird Measurer (Assessment)", 1.97, float("nan"))))))


###  Time in current playing session

In [28]:
# can just use the game_time column

### Has user played assessment before?

### How many times they took an assessment

In [29]:
#train_2 = train.merge(train_1, left_index=True, right_index=True)

In [30]:
#train_1 = train.groupby(["installation_id","title_y"]).transform('count')


In [31]:
#train_1= train_1[["event_id"]]

In [32]:
#train_1.rename(columns={"event_id":"times_played"}, inplace=True)

In [33]:
def times_took_assess(df):
    df_1 = df.groupby(["installation_id","title_y"]).transform('count')
    df_2= df_1[["event_id"]]
    df_2 = df_2.rename(columns={"event_id":"times_played"})
    df = df.merge(df_2, left_index=True, right_index=True)
    return df["times_played"].iloc[-1]

build features

In [36]:
train["assessment"].dtype

dtype('float64')

In [40]:
%%time

count = 0
X, y = [], []
for i, row in train_cuts.iterrows():
    count += 1
    installation_id, game_session = row['installation_id'], row['game_session']
    df = train[train['installation_id'] == installation_id]
    # get the timestamp of the cut row
    cut_time = df.loc[i,'timestamp']
    # cut the df
    df = df[df['timestamp'] <= cut_time]
    # cut off last row (assumes df is sorted by time)
    df = df.iloc[:-1,:]
    if df.empty:
        continue
    df['accuracy_group'] = row['accuracy_group']
    
    feature = {'part_of_day':max(df["segment_of_day"].iloc[-1], 0),
               'assessment_taken':max(df["assessment"].iloc[-1], 0),                
               'time_playing_for':max(df["game_time"].iloc[-1], 0),
               'playtime_vs_avg': time_compared_to_normal(df),
               'times_took_asses': times_took_assess(df),
               '2010':max(df[2010].iloc[-1], 0),
               '2020':max(df[2020].iloc[-1], 0),
               '2025':max(df[2025].iloc[-1], 0),
               '2030':max(df[2030].iloc[-1], 0),
               '2035':max(df[2035].iloc[-1], 0),
               '3010':max(df[3010].iloc[-1], 0),
               '3020':max(df[3020].iloc[-1], 0),
               '3021':max(df[3021].iloc[-1], 0),
               '3110':max(df[3110].iloc[-1], 0),
               '3120':max(df[3120].iloc[-1], 0),
               '3121':max(df[3121].iloc[-1], 0),
               '4020':max(df[4020].iloc[-1], 0),
               '4025':max(df[4025].iloc[-1], 0),
               '4030':max(df[4030].iloc[-1], 0),
               '4035':max(df[4035].iloc[-1], 0),
               '4040':max(df[4040].iloc[-1], 0),
               '4070':max(df[4070].iloc[-1], 0),
               '4080':max(df[4080].iloc[-1], 0),
               '4090':max(df[4090].iloc[-1], 0),
               '4100':max(df[4100].iloc[-1], 0),
               '4110':max(df[4110].iloc[-1], 0)}
    
    X.append(feature)
    y.append(row['accuracy_group'])
    
    if count % 1000 == 0:
        print('progress = {}%'.format(count/17690*100))
    
    #if count > 2:  # note we're just making features for the first n cuts
    #    break

X = pd.DataFrame(X)
X.shape, len(y)

progress = 5.652911249293386%
progress = 11.305822498586773%
progress = 16.95873374788016%
progress = 22.611644997173546%
progress = 28.26455624646693%
progress = 33.91746749576032%
progress = 39.5703787450537%
progress = 45.22328999434709%
progress = 50.87620124364047%
progress = 56.52911249293386%
progress = 62.182023742227244%
progress = 67.83493499152064%
progress = 73.48784624081401%
progress = 79.1407574901074%
progress = 84.7936687394008%
progress = 90.44657998869418%
progress = 96.09949123798756%
Wall time: 5h 24min 28s


note: there is some loss here (users with no data) and we need to figure out how to handle these cases

In [41]:
X

,2010,2020,2025,2030,2035,3010,3020,3021,3110,3120,...,4070,4080,4090,4100,4110,assessment_taken,part_of_day,playtime_vs_avg,time_playing_for,times_took_asses
0,4,112,12,97,8,620,34,89,610,34,...,592,0,4,13,14,NaN,1.524239,0 -0.160742 1 -0.160742 2 -0.16074...,0,NaN
1,4,112,12,97,8,620,34,89,610,34,...,592,0,4,13,14,NaN,1.524239,0 -0.090989 1 -0.090989 2 -0.09...,104816,NaN
2,4,112,12,97,8,620,34,89,610,34,...,592,0,4,13,14,1.14,1.524239,0 -0.084553 1 -0.084553 2 -0.08...,92799,87.0
3,4,112,12,97,8,620,34,89,610,34,...,592,0,4,13,14,NaN,1.502395,0 -12.617415 1 -12.617415 2 -12...,8789,NaN
4,4,112,12,97,8,620,34,89,610,34,...,592,0,4,13,14,NaN,1.502395,0 -10.534029 1 -10.534029 2 -10...,80899,NaN
5,0,52,2,45,2,251,29,22,245,29,...,676,0,4,6,2,NaN,1.556186,0 -2.169261 1 -2.169261 2 -2.16...,0,NaN
6,0,52,2,45,2,251,29,22,245,29,...,676,0,4,6,2,NaN,1.524239,0 11.498946 1 11.498946 2 11...,0,NaN
7,0,52,2,45,2,251,29,22,245,29,...,676,0,4,6,2,NaN,1.524239,0 10.793769 1 10.793769 2 10...,11949,NaN
8,1,21,1,19,0,184,0,1,183,0,...,64,6,12,1,1,NaN,1.524239,"0 0.0 1 0.0 2 0.0 Name: mean, dtype: ...",41,NaN
9,3,52,2,44,2,124,39,34,121,38,...,186,0,2,9,5,NaN,1.556186,0 0.0 1 0.0 2 0.0 3 0.0 4 0.0 5...,0,NaN


In [45]:
X.to_csv(r'C:\Users\593939\Desktop\DSB 2019\features_Sheehan.csv')

In [47]:
X

,2010,2020,2025,2030,2035,3010,3020,3021,3110,3120,...,4070,4080,4090,4100,4110,assessment_taken,part_of_day,playtime_vs_avg,time_playing_for,times_took_asses
0,4,112,12,97,8,620,34,89,610,34,...,592,0,4,13,14,NaN,1.524239,0 -0.160742 1 -0.160742 2 -0.16074...,0,NaN
1,4,112,12,97,8,620,34,89,610,34,...,592,0,4,13,14,NaN,1.524239,0 -0.090989 1 -0.090989 2 -0.09...,104816,NaN
2,4,112,12,97,8,620,34,89,610,34,...,592,0,4,13,14,1.14,1.524239,0 -0.084553 1 -0.084553 2 -0.08...,92799,87.0
3,4,112,12,97,8,620,34,89,610,34,...,592,0,4,13,14,NaN,1.502395,0 -12.617415 1 -12.617415 2 -12...,8789,NaN
4,4,112,12,97,8,620,34,89,610,34,...,592,0,4,13,14,NaN,1.502395,0 -10.534029 1 -10.534029 2 -10...,80899,NaN
5,0,52,2,45,2,251,29,22,245,29,...,676,0,4,6,2,NaN,1.556186,0 -2.169261 1 -2.169261 2 -2.16...,0,NaN
6,0,52,2,45,2,251,29,22,245,29,...,676,0,4,6,2,NaN,1.524239,0 11.498946 1 11.498946 2 11...,0,NaN
7,0,52,2,45,2,251,29,22,245,29,...,676,0,4,6,2,NaN,1.524239,0 10.793769 1 10.793769 2 10...,11949,NaN
8,1,21,1,19,0,184,0,1,183,0,...,64,6,12,1,1,NaN,1.524239,"0 0.0 1 0.0 2 0.0 Name: mean, dtype: ...",41,NaN
9,3,52,2,44,2,124,39,34,121,38,...,186,0,2,9,5,NaN,1.556186,0 0.0 1 0.0 2 0.0 3 0.0 4 0.0 5...,0,NaN


In [236]:
def time_compared_to_normal(df):
    a = df.groupby("installation_id").describe()
    avg_time = pd.DataFrame(data=a["time_of_day"]["mean"])
    avg_time.reset_index(level=0, inplace=True)
    df = pd.merge(df, avg_time, on=['installation_id'], how='left')
    # calculate the "normal" time for this user to play
    return df["mean"] - df.iloc[-1,'time_of_day'] 

In [ ]:
def 

In [ ]:
len(X), len(y)

K-Fold Cross Validation

In [122]:
from sklearn import tree
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold

k_folds = 5
skf = StratifiedKFold(n_splits=k_folds, shuffle=True)

X, y = np.array(X), np.array(y)
total_runs = skf.get_n_splits()
scores = []
count = 0
for train_index, test_index in skf.split(X, y):
    count += 1
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(X_train, y_train)
    score = cohen_kappa_score(clf.predict(X_test), y_test, weights='quadratic')
    scores.append(score)
    print('Run {}/{} -- kappa_score: {}'.format(count, total_runs, score))
print('\nmean score: {}'.format(score)) 

ValueError: Cannot have number of splits n_splits=5 greater than the number of samples: 2.

## Feature Ideas

- mean time in an assessment (exclude outliers?)
- cumulative time playing the game?
- installation duration mean (what does this mean? same as above?)
- accuracy of game/activity directly prior to the assessment
- has it been a long gaming session?


- "exit_type":"game_completed" -- if a player has any other exit types, they are a bad player
- event_id=cdd22e43 -- this event could show a player is unskilled